In [ ]:
import pandas as pd
import numpy as np

import os
import glob

In [ ]:
from transformers import pipeline, AutoTokenizer
from google.colab import drive

In [ ]:
!pip install -q wrds psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00


In [ ]:
import datetime as dt

In [ ]:
from datasets import load_dataset
import random
from datetime import datetime, timedelta

import wrds
import re

# 1. Acquire FINRA Short Interest Data

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 2: Set your actual Google Drive path (update as needed)
folder_path = '/content/drive/MyDrive/Projects/JPM Short Interest/Data/Short Interest FINRA'

# Step 3: Read all matching CSV files
csv_files = glob.glob(os.path.join(folder_path, 'shrt*.csv'))

# Step 4: Load and concatenate
finra_df_list = []
for file in csv_files:
    finra_df = pd.read_csv(file, sep='|', dtype=str, engine='python', on_bad_lines='skip')
    finra_df_list.append(finra_df)

finra_df = pd.concat(finra_df_list, ignore_index=True)

In [ ]:
# Data Cleaning
finra_df_cleaned = finra_df[finra_df['marketClassCode'] != '']

# Show preview
finra_df_cleaned.shape

(2197833, 14)

In [ ]:
AAPL = finra_df_cleaned[finra_df_cleaned["symbolCode"] == "AAPL"]
AAPL.head()

,accountingYearMonthNumber,symbolCode,issueName,issuerServicesGroupExchangeCode,marketClassCode,currentShortPositionQuantity,previousShortPositionQuantity,stockSplitFlag,averageDailyVolumeQuantity,daysToCoverQuantity,revisionFlag,changePercent,changePreviousNumber,settlementDate
48,20250731,AAPL,Apple Inc. Common Stock,R,NNM,104634941,93946599,NaN,48911215,2.14,NaN,11.38,10688342,2025-07-31
20642,20250715,AAPL,Apple Inc. Common Stock,R,NNM,93946599,110101153,NaN,48526568,1.94,NaN,-14.67,-16154554,2025-07-15
41249,20240731,AAPL,Apple Inc. Common Stock,R,NNM,117696224,135383184,NaN,48246946,2.44,NaN,-13.06,-17686960,2024-07-31
60777,20240715,AAPL,Apple Inc. Common Stock,R,NNM,135383184,132235437,NaN,55946376,2.42,NaN,2.38,3147747,2024-07-15
80351,20230731,AAPL,Apple Inc. Common Stock,R,NNM,105460144,126626604,NaN,52323234,2.02,NaN,-16.72,-21166460,2023-07-31


In [ ]:
AAPL.shape

(110, 14)

# 2. Acquire Transcript Data from WRDS

In [ ]:
# 1. Connect and discover transcript tables

db = wrds.Connection()  # opens browser auth the first time

# See which library holds transcripts on your site (often 'ciq')
libs = db.list_libraries()
print(libs)

# List candidate tables and pick ones with "transcript" in the name
ciq_tables = db.list_tables(library='ciq')  # try 'ciq' first; if not, print(libs) and try the vendor lib that exists for you
[t for t in ciq_tables if re.search('transcript', t, re.I)]

Enter your WRDS username [root]:beihann2
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
['aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', 'audit_common', 'audit_corp_legal', 'auditsmp', 'auditsmp_all', 'bank', 'bank_all', 'bank_premium_samp', 'banksamp', 'block', 'block_all', 'boardex', 'boardex_na', 'boardex_trial', 'boardsmp', 'bvd_amadeus_trial', 'bvd_bvdbankf_trial', 'bvd_orbis_trial', 'bvdsamp', 'calcbench_trial', 'calcbnch', 'candid_samp', 'cboe', 'cboe_all', 'cboe_sample', 'cboesamp', 'cddsamp', 'ciq', 'ciq_common', 'ciq_transcripts', 'ciqsamp', 'ciqsamp_capstrct', 'ciqsamp_common', 'ciqsamp_keydev', 'ciqsamp_pplintel', 'ciqsamp_ratings', 'ciqsamp_transactions', 'ciqsamp_transcripts', 'cisdmsmp', 'columnar', 'comp', 'comp_bank_daily', 'comp_execucomp', 'comp_global_d

['ciqtranscript',
 'ciqtranscriptcollectiontype',
 'ciqtranscriptcomponent',
 'ciqtranscriptcomponenttype',
 'ciqtranscriptdelayreason',
 'ciqtranscriptdelayreasontype',
 'ciqtranscriptperson',
 'ciqtranscriptpresentationtype',
 'ciqtranscriptspeakertype',
 'wrds_transcript_detail',
 'wrds_transcript_person']

In [ ]:
db.describe_table(library='ciq_transcripts', table='wrds_transcript_detail')
db.describe_table(library='ciq_transcripts', table='wrds_transcript_person')

Approximately 1739865 rows in ciq_transcripts.wrds_transcript_detail.
Approximately 87805852 rows in ciq_transcripts.wrds_transcript_person.


,name,nullable,type,comment
0,transcriptid,True,"NUMERIC(11, 0)",Transcript ID
1,transcriptcomponentid,True,"NUMERIC(11, 0)",Transcript Component ID
2,componentorder,True,INTEGER,Transcript Component Order
3,transcriptcomponenttypeid,True,INTEGER,Transcript Component Type ID
4,transcriptcomponenttypename,True,VARCHAR(400),Transcript Component Type Name
5,transcriptpersonid,True,"NUMERIC(11, 0)",Transcript Person ID
6,transcriptpersonname,True,VARCHAR(800),Name of Person
7,proid,True,"NUMERIC(11, 0)",Professional ID
8,companyofperson,True,VARCHAR(800),Company of Person
9,speakertypeid,True,INTEGER,Speaker Type ID


In [ ]:
# STEP 1: inspect columns + sample rows
cols = db.describe_table('ciq_transcripts', 'ciqtranscript')
print(cols)

sample = db.raw_sql("SELECT * FROM ciq_transcripts.ciqtranscript LIMIT 5;")
print(sample.columns.tolist())
sample.head()

Approximately 1669810 rows in ciq_transcripts.ciqtranscript.
                           name  nullable            type  \
0                      keydevid      True  NUMERIC(11, 0)   
1                  transcriptid      True  NUMERIC(11, 0)   
2    transcriptcollectiontypeid      True         INTEGER   
3  transcriptpresentationtypeid      True         INTEGER   
4     transcriptcreationdateutc      True       TIMESTAMP   
5                audiolengthsec      True  NUMERIC(11, 0)   

                        comment  
0                      keydevid  
1                  transcriptid  
2    transcriptcollectiontypeid  
3  transcriptpresentationtypeid  
4     transcriptcreationdateutc  
5                audiolengthsec  
['keydevid', 'transcriptid', 'transcriptcollectiontypeid', 'transcriptpresentationtypeid', 'transcriptcreationdateutc', 'audiolengthsec']


,keydevid,transcriptid,transcriptcollectiontypeid,transcriptpresentationtypeid,transcriptcreationdateutc,audiolengthsec
0,0.0,2745244.0,13,5,2023-02-20 16:25:24,<NA>
1,7.0,864627.0,6,5,2015-09-08 08:56:36,<NA>
2,202048.0,16557.0,6,5,2009-02-11 03:03:26,<NA>
3,1018019.0,708.0,6,5,2008-06-20 00:11:20,<NA>
4,1026789.0,4672.0,6,5,2008-06-20 00:51:51,<NA>


In [ ]:
params = {
    "start": "2023-08-01",
    "end":   "2025-08-01",
}

sql_multi = """
WITH patterns(ticker, kw) AS (
  VALUES
    ('AAPL',  '%%Apple%%'),
    ('MSFT',  '%%Microsoft%%'),
    ('AMZN',  '%%Amazon%%'),
    ('GOOGL', '%%Google%%'),
    ('META',  '%%Meta%%'),
    ('TSLA',  '%%Tesla%%')
),
calls AS (
  SELECT
      t.transcriptid,
      t.transcriptcreationdateutc AS event_dt
  FROM ciq_transcripts.ciqtranscript t
  WHERE t.transcriptcreationdateutc BETWEEN %(start)s AND %(end)s
),
joined AS (
  SELECT
      c.transcriptid,
      string_agg(c.componenttext, E'\n' ORDER BY c.componentorder) AS full_text
  FROM ciq_transcripts.ciqtranscriptcomponent c
  WHERE c.transcriptid IN (SELECT transcriptid FROM calls)
  GROUP BY c.transcriptid
),
-- match each transcript text to all patterns that hit
hits AS (
  SELECT
      j.transcriptid,
      p.ticker,
      j.full_text
  FROM joined j
  JOIN patterns p
    ON j.full_text ILIKE p.kw
),
-- if a transcript matched multiple patterns (rare but possible), keep one row per transcript
dedup AS (
  SELECT DISTINCT ON (h.transcriptid)
      h.transcriptid,
      h.ticker,
      j.full_text
  FROM hits h
  JOIN joined j USING (transcriptid)
  ORDER BY h.transcriptid, h.ticker  -- deterministic pick
)
SELECT
    c.transcriptid,
    c.event_dt,
    d.ticker,
    d.full_text
FROM calls c
JOIN dedup d USING (transcriptid)
ORDER BY c.event_dt;
"""

tx_multi = db.raw_sql(sql_multi, params=params)
print(tx_multi.shape)
tx_multi.head(3)

(67695, 4)


,transcriptid,event_dt,ticker,full_text
0,2863440.0,2023-08-01 00:13:49,AMZN,"All right. Good morning, everybody here in Nas..."
1,2863451.0,2023-08-01 01:09:56,AAPL,"Good morning, everyone, and welcome to Ryman H..."
2,2863460.0,2023-08-01 01:56:12,MSFT,"Good day, and thank you for standing by. Welco..."


In [ ]:
tx = tx_multi.copy()

In [ ]:
# 1) Per-ticker keyword dictionaries (extend as you like)
TICKER_PATTERNS = {
    "AAPL": {
        "pos": [
            r"\bapple inc\b", r"\bapple\b", r"\bnasdaq:\s*aapl\b", r"\baapl\b",
            r"\bticker\s*aapl\b", r"\btim cook\b", r"\bluca maestri\b"
        ],
        "neg": [
            r"\bryman\b", r"\bmarriott\b", r"\bhyatt\b",
            r"\bdatadog\b", r"\bokta\b", r"\bcrowdstrike\b", r"\btenable\b",
            r"\bmicrosoft ignite\b", r"\brsa conference\b"
        ]
    },
    "MSFT": {
        "pos": [r"\bmicrosoft\b", r"\bmsft\b", r"\bnasdaq:\s*msft\b", r"\bsatya nadella\b", r"\bamy hood\b"],
        "neg": [r"\bapple\b", r"\baapl\b"]  # light example; tailor as needed
    },
    "AMZN": {
        "pos": [r"\bamazon\.?com?\b", r"\bamzn\b", r"\bnasdaq:\s*amzn\b", r"\bandy jassy\b"],
        "neg": []
    },
    "GOOGL": {
        "pos": [r"\balphabet\b", r"\bgoogle\b", r"\bgoogl\b", r"\bnasdaq:\s*googl\b", r"\bsundar pichai\b", r"\bruth porat\b"],
        "neg": []
    },
    "META": {
        "pos": [r"\bmeta platforms?\b", r"\bfacebook\b", r"\bmeta\b", r"\bmeta platforms inc\b", r"\bmeta platforms,? inc\b", r"\bnasdaq:\s*meta\b"],
        "neg": []
    },
    "TSLA": {
        "pos": [r"\btesla\b", r"\btsla\b", r"\bnasdaq:\s*tsla\b", r"\belon musk\b"],
        "neg": []
    }
}

# 2) Build a predicate for a given ticker
def make_is_company_text(ticker: str):
    pats = TICKER_PATTERNS.get(ticker.upper(), None)
    if pats is None:
        raise ValueError(f"No patterns registered for ticker '{ticker}'. Add to TICKER_PATTERNS.")

    pos_res = [re.compile(p, re.IGNORECASE) for p in pats["pos"]]
    neg_res = [re.compile(n, re.IGNORECASE) for n in pats["neg"]]

    def _pred(s: str) -> bool:
        if not isinstance(s, str) or not s.strip():
            return False
        # must hit at least one positive
        if not any(r.search(s) for r in pos_res):
            return False
        # and not hit any exclusion
        if any(r.search(s) for r in neg_res):
            return False
        return True

    return _pred

# 3) Helper to filter a df by ticker using the predicate
def filter_transcripts_by_ticker(df: pd.DataFrame, ticker_col: str = None, text_col: str = "full_text", ticker: str = "AAPL"):
    """
    If df already has a ticker column (from your multi-ticker SQL), it filters by that *and* text predicate.
    If not, it uses only the text predicate to infer likely matches for the chosen ticker.
    """
    pred = make_is_company_text(ticker)
    if ticker_col and ticker_col in df.columns:
        sub = df[df[ticker_col].str.upper() == ticker.upper()].copy()
        return sub[sub[text_col].apply(pred)]
    else:
        return df[df[text_col].apply(pred)].copy()
# 1) Filter transcripts for a basket of tickers (reuses your helper)
tickers = ["AAPL","MSFT","GOOGL","AMZN","META","TSLA"]

tx_dict = {}
for tk in tickers:
    sub = filter_transcripts_by_ticker(tx_multi, ticker_col="ticker", text_col="full_text", ticker=tk)
    # ensure event_dt is datetime
    sub["event_dt"] = pd.to_datetime(sub["event_dt"], errors="coerce")
    # keep a consistent set of columns
    tx_dict[tk] = sub[["transcriptid","event_dt","ticker","full_text"]].dropna(subset=["event_dt"])

# Combine all tickers
tx_filt = pd.concat(tx_dict.values(), ignore_index=True)
print("Combined shape:", tx_filt.shape)


Combined shape: (25622, 4)


In [ ]:
# A) Global anchor + 14-day bins for transcripts
tx_filt = tx_filt.copy()
tx_filt["event_dt"] = pd.to_datetime(tx_filt["event_dt"], errors="coerce")
tx_filt = tx_filt.dropna(subset=["event_dt", "ticker"])

anchor = tx_filt["event_dt"].min()
tx_filt["interval"] = ((tx_filt["event_dt"] - anchor).dt.days // 14).astype(int)

# one transcript per (ticker, interval): most recent in that bin
by_bin_tx = (
    tx_filt.sort_values(["ticker", "event_dt"])
           .groupby(["ticker", "interval"], as_index=False)
           .tail(1)[["ticker","interval","event_dt","transcriptid","full_text"]]
           .sort_values(["ticker","interval"])
           .reset_index(drop=True)
)

print("Transcripts by-bin:", by_bin_tx.shape)
by_bin_tx.head()

Transcripts by-bin: (313, 5)


,ticker,interval,event_dt,transcriptid,full_text
0,AAPL,0,2023-08-15 01:10:12,2884277.0,[Audio Gap] [Operator Instructions] Just a re...
1,AAPL,1,2023-08-28 17:00:47,2896423.0,"Friends from the investment sector, greetings...."
2,AAPL,2,2023-09-11 20:56:28,2906074.0,[Presentation] Kepler Day 2023. Good morning o...
3,AAPL,3,2023-09-24 06:06:47,2911674.0,"Hello, ladies and gentlemen. Welcome to the co..."
4,AAPL,4,2023-10-09 17:47:12,2915071.0,"Good morning, everyone. Thanks for joining us...."


In [ ]:
# B) FINRA prep — align to the same anchor and 14-day grid
finra = finra_df.copy()  # <-- rename if needed
finra["settlementDate"] = pd.to_datetime(finra["settlementDate"], errors="coerce")
finra = finra.dropna(subset=["settlementDate"])

# harmonize ticker column name to match transcripts
finra = finra.rename(columns={"symbolCode": "ticker"})

# use the SAME anchor as transcripts
finra["interval"] = ((finra["settlementDate"] - anchor).dt.days // 14).astype(int)

# one FINRA row per (ticker, interval): latest settlement in that bin
by_bin_fi = (
    finra.sort_values(["ticker","settlementDate"])
         .groupby(["ticker","interval"], as_index=False)
         .tail(1)
)

print("FINRA by-bin:", by_bin_fi.shape)
by_bin_fi.head()

FINRA by-bin: (2178299, 15)


,accountingYearMonthNumber,ticker,issueName,issuerServicesGroupExchangeCode,marketClassCode,currentShortPositionQuantity,previousShortPositionQuantity,stockSplitFlag,averageDailyVolumeQuantity,daysToCoverQuantity,revisionFlag,changePercent,changePreviousNumber,settlementDate,interval
2019402,20210115,A,Agilent Technologies Inc.,A,NYSE,2266746,2502573,NaN,1618485,1.40,NaN,-9.42,-235827,2021-01-15,-67
2000454,20210129,A,Agilent Technologies Inc.,A,NYSE,2272817,2266746,NaN,1446062,1.57,NaN,0.27,6071,2021-01-29,-66
1981098,20210212,A,Agilent Technologies Inc.,A,NYSE,2352823,2272817,NaN,1368962,1.72,NaN,3.52,80006,2021-02-12,-65
1961603,20210226,A,Agilent Technologies Inc.,A,NYSE,2380017,2352823,NaN,1816473,1.31,NaN,1.16,27194,2021-02-26,-64
1601381,20210315,A,Agilent Technologies Inc.,A,NYSE,2908967,2380017,NaN,1784590,1.63,NaN,22.22,528950,2021-03-15,-63


In [ ]:
merged = (
    by_bin_tx.merge(by_bin_fi, on=["ticker","interval"], how="inner")
             .sort_values(["ticker","interval"])
             .reset_index(drop=True)
)

print("Merged shape:", merged.shape)
merged.head(3)

Merged shape: (283, 18)


,ticker,interval,event_dt,transcriptid,full_text,accountingYearMonthNumber,issueName,issuerServicesGroupExchangeCode,marketClassCode,currentShortPositionQuantity,previousShortPositionQuantity,stockSplitFlag,averageDailyVolumeQuantity,daysToCoverQuantity,revisionFlag,changePercent,changePreviousNumber,settlementDate
0,AAPL,0,2023-08-15 01:10:12,2884277.0,[Audio Gap] [Operator Instructions] Just a re...,20230815,Apple Inc. Common Stock,R,NNM,88852352,105460144,NaN,62061606,1.43,NaN,-15.75,-16607792,2023-08-15
1,AAPL,2,2023-09-11 20:56:28,2906074.0,[Presentation] Kepler Day 2023. Good morning o...,20230831,Apple Inc. Common Stock,R,NNM,102091714,88851550,NaN,53191009,1.92,R,14.90,13240164,2023-08-31
2,AAPL,3,2023-09-24 06:06:47,2911674.0,"Hello, ladies and gentlemen. Welcome to the co...",20230915,Apple Inc. Common Stock,R,NNM,107325921,102091714,NaN,75396415,1.42,NaN,5.13,5234207,2023-09-15


In [ ]:
# 0) (Re)install parquet support if needed
!pip install -q pyarrow

# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1) NLTK setup (run once per session)
import nltk
nltk.download('punkt')
# some newer NLTK installs also need this:
try:
    nltk.download('punkt_tab')
except:
    pass

from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np

# 2) Your function (use a conservative max_words to stay <512 tokens)
def chunk_text(text, max_words=220):
    if not isinstance(text, str) or not text.strip():
        return []
    sents = sent_tokenize(text)
    chunks, current, count = [], [], 0
    for sent in sents:
        wlen = len(word_tokenize(sent))
        if count + wlen > max_words:
            if current:
                chunks.append(" ".join(current))
            current = [sent]
            count = wlen
        else:
            current.append(sent)
            count += wlen
    if current:
        chunks.append(" ".join(current))
    return chunks

# 3) Build a document table and a chunk table
docs = merged.copy()                       # merged: one row per interval
docs["doc_id"] = np.arange(len(docs))
rows = []
for doc_id, txt in zip(docs["doc_id"], docs["full_text"].fillna("")):
    parts = chunk_text(txt, max_words=220)
    for j, ch in enumerate(parts):
        rows.append({
            "doc_id": doc_id,
            "chunk_id": j,
            "chunk_text": ch,
            "chunk_word_len": len(ch.split())
        })
chunks = pd.DataFrame(rows)

# 4) Save for scoring in NoLBERT notebook
OUT_DIR = "/content/drive/MyDrive/Projects/JPM Short Interest/outputs"
docs.to_parquet(f"{OUT_DIR}/merged_docs.parquet", index=False)
chunks.to_parquet(f"{OUT_DIR}/merged_chunks_nltk.parquet", index=False)
print("Saved:\n ", f"{OUT_DIR}/merged_docs.parquet", "\n ", f"{OUT_DIR}/merged_chunks_nltk.parquet")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Saved:
  /content/drive/MyDrive/Projects/JPM Short Interest/outputs/merged_docs.parquet 
  /content/drive/MyDrive/Projects/JPM Short Interest/outputs/merged_chunks_nltk.parquet
